In [4]:
import os
import base64
import tempfile
from pathlib import Path
from pdf2image import convert_from_path
import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI


# Load OpenAI API key from .env file
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ Vision Prompt


vision_prompt = """
This image is from the first few pages of a corporate report.

Please identify the time period that the report covers, based on any visible text, charts, or tables, body text, or headers, footer, or any other visible elements in the image
As much as you can, you can also finf the mounth and date.


Return examples like:
- “April 2023 – March 2024”
- “Fiscal Year Ending 31 March 2023”
- “2022”
- "31 December 2022 to 30 June 2023"

If no such date range is visible, respond with “NOT FOUND”.
Return only the most specific date range or fiscal year visible.
"""




def encode_image_to_base64(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def call_gpt4_vision(image_path):
    b64_image = encode_image_to_base64(image_path)
    try:
        response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": [
                {"type": "text", "text": vision_prompt},
                {"type": "image_url", "image_url": {
                    "url": f"data:image/png;base64,{b64_image}"
                }}
            ]}
        ],
        max_tokens=300
    )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"ERROR: {e}"

def extract_report_year_from_pdf(pdf_path, max_pages=3):
    with tempfile.TemporaryDirectory() as tmpdir:
        try:
            images = convert_from_path(pdf_path, dpi=300, first_page=1, last_page=max_pages, output_folder=tmpdir)
            for img in images:
                img_path = Path(tmpdir) / f"{Path(pdf_path).stem}.png"
                img.save(img_path)
                result = call_gpt4_vision(img_path)
                if "not found" not in result.lower():
                    return result
        except Exception as e:
            return f"ERROR: {e}"
    return "NOT FOUND"

def batch_process_pdf_folder(pdf_folder, output_csv, max_pages=5):
    results = []

    for filename in tqdm(os.listdir(pdf_folder)):
        if not filename.lower().endswith(".pdf"):
            continue
        pdf_path = os.path.join(pdf_folder, filename)
        result = extract_report_year_from_pdf(pdf_path, max_pages)
        results.append({
            "filename": filename,
            "report_year_vision": result
        })

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"✅ Saved results to {output_csv}")


In [5]:
if __name__ == "__main__":
    folder_path = "pdf_folder"  # ⚠️ 替换成你本地PDF文件夹路径
    output_csv = "output/report_years_extracted_new.csv"
    batch_process_pdf_folder(folder_path, output_csv, max_pages=3)


 20%|█▉        | 252/1279 [35:44<1:50:09,  6.44s/it] /opt/homebrew/Caskroom/miniconda/base/envs/irp_pdf/lib/python3.12/site-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (136875639 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 67%|██████▋   | 855/1279 [5:05:54<41:20,  5.85s/it]      /opt/homebrew/Caskroom/miniconda/base/envs/irp_pdf/lib/python3.12/site-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (151052078 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 1279/1279 [5:47:13<00:00, 16.29s/it] 

✅ Saved results to output/report_years_extracted_new.csv


In [3]:
import pandas as pd

# 读取文件
anno_df = pd.read_excel("check/rfyear_annotation.xlsx")
result_df = pd.read_csv("output/full_pipeline_results.csv")

# 清洗文件名列
anno_df["pdf_name_clean"] = anno_df["pdf_name"].str.strip().str.lower()
result_df["filename_clean"] = result_df["filename"].str.strip().str.lower()

# 合并两个表
merged_df = pd.merge(
    anno_df,
    result_df,
    left_on="pdf_name_clean",
    right_on="filename_clean",
    how="left"
)

# 年份对比函数
def match_year(human, gpt):
    if pd.isna(human) or pd.isna(gpt):
        return "missing"
    return "match" if str(human).strip().lower() == str(gpt).strip().lower() else "mismatch"

# 添加匹配结果
merged_df["match_result"] = merged_df.apply(
    lambda row: match_year(row["chosen_rfyear"], row["report_year_vision"]),
    axis=1
)

# 仅保留有用字段
final_df = merged_df[[
    "pdf_name", "chosen_rfyear", "report_year_vision", "match_result"
]]

# 保存结果
final_df.to_csv("rfyear_comparison_results_minimal.csv", index=False)

# 输出统计
print("✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv")
print("📊 匹配情况统计：")
print(final_df["match_result"].value_counts())

✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv
📊 匹配情况统计：
match_result
missing     386
mismatch    137
match        78
Name: count, dtype: int64


In [6]:
import pandas as pd

# 读取文件
anno_df = pd.read_excel("check/rfyear_annotation.xlsx")
result_df = pd.read_csv("output/report_years_extracted_new.csv")

# 清洗文件名列
anno_df["pdf_name_clean"] = anno_df["pdf_name"].str.strip().str.lower()
result_df["filename_clean"] = result_df["filename"].str.strip().str.lower()

# 合并两个表
merged_df = pd.merge(
    anno_df,
    result_df,
    left_on="pdf_name_clean",
    right_on="filename_clean",
    how="left"
)

# 年份对比函数
def match_year(human, gpt):
    if pd.isna(human) or pd.isna(gpt):
        return "missing"
    return "match" if str(human).strip().lower() == str(gpt).strip().lower() else "mismatch"

# 添加匹配结果
merged_df["match_result"] = merged_df.apply(
    lambda row: match_year(row["chosen_rfyear"], row["report_year_vision"]),
    axis=1
)

# 仅保留有用字段
final_df = merged_df[[
    "pdf_name", "chosen_rfyear", "report_year_vision", "match_result"
]]

# 保存结果
final_df.to_csv("rfyear_comparison_results_minimal.csv", index=False)

# 输出统计
print("✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv")
print("📊 匹配情况统计：")
print(final_df["match_result"].value_counts())

✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv
📊 匹配情况统计：
match_result
missing     386
mismatch    215
Name: count, dtype: int64


In [3]:
import os
import fitz  # PyMuPDF
import openai
import pytesseract
import tempfile
import base64
import json
from pdf2image import convert_from_path
from PIL import Image
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI


# Load OpenAI API key from .env file
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ Vision Prompt
openai.api_key = "sk-..."  # ← 请替换为你的 API key

def extract_front_back_text(pdf_path, front_n=20, back_n=20):
    try:
        doc = fitz.open(pdf_path)
        texts = [doc[i].get_text() for i in range(min(front_n, len(doc)))]
        texts += [doc[i].get_text() for i in range(max(0, len(doc) - back_n), len(doc))]
        doc.close()
        full_text = "\n".join(texts)
        if len(full_text.strip()) < 50:
            raise ValueError("Too little text, fallback to OCR.")
        return full_text
    except:
        return extract_text_with_ocr(pdf_path)
    

def extract_text_with_ocr(pdf_path, dpi=300):
    with tempfile.TemporaryDirectory() as path:
        images = convert_from_path(pdf_path, dpi=dpi, output_folder=path)
        text_parts = [pytesseract.image_to_string(img) for img in images[:3] + images[-3:]]
        return "\n".join(text_parts)
    
def build_report_year_prompt(text):
    return f"""
You are an assistant helping extract the reporting year or fiscal year from the following corporate report excerpt.

1. Look for expressions such as:
   - "For the year ended 31 March 2022"
   - "Reporting period: April 2021 – March 2022"
   - "FY2020", etc.
2. If found, return this JSON:
{{
  "report_year": "April 2021 – March 2022",
  "source": "Page 2, main text"
}}

If not found:
{{
  "report_year": null,
  "source": "NOT FOUND"
}}

Report text:
{text}
"""


def encode_image_to_base64(pil_image):
    with tempfile.NamedTemporaryFile(suffix=".png") as f:
        pil_image.save(f.name, format="PNG")
        with open(f.name, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
        
def extract_year_from_vision(pdf_path, client, page_limit=3):
    images = convert_from_path(pdf_path, dpi=300)
    for i, img in enumerate(images[:page_limit]):
        b64 = encode_image_to_base64(img)
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": [
                    {"type": "text", "text": f"Please extract the fiscal year or reporting period from page {i+1}."},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{b64}"}}
                ]}
            ],
            max_tokens=300
        )
        result = response.choices[0].message.content
        if "not found" not in result.lower():
            return {"report_year": result.strip(), "source": f"Vision page {i+1}"}
    return {"report_year": None, "source": "Vision NOT FOUND"}

def extract_report_year(pdf_path, client):
    text = extract_front_back_text(pdf_path)
    prompt = build_report_year_prompt(text)
    try:
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=300
        )
        content = response.choices[0].message.content
        parsed = json.loads(content) if content.startswith("{") else eval(content)
        if parsed["report_year"]:
            return parsed
    except:
        pass
    return extract_year_from_vision(pdf_path, client)

In [4]:
from pathlib import Path
import pandas as pd

pdf_dir = "pdf_folder"  # 替换为你的文件夹路径
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


results = []
for pdf in tqdm(Path(pdf_dir).glob("*.pdf")):
    try:
        out = extract_report_year(str(pdf), client)
        out["filename"] = pdf.name
        results.append(out)
    except Exception as e:
        results.append({"filename": pdf.name, "report_year": None, "source": f"ERROR: {e}"})

df = pd.DataFrame(results)
df.to_csv("results/extracted_report_years_mini.csv", index=False)

256it [11:29,  1.59s/it]

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed



724it [36:16,  2.99s/it]

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: obje

835it [40:33,  2.04s/it]

: 

In [ ]:
import os
import fitz  # PyMuPDF
import openai
import pytesseract
import tempfile
import base64
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from tqdm import tqdm
from pdf2image import convert_from_path
from PIL import Image
from openai import OpenAI

# ========== 1. 初始化 OpenAI ==========
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ========== 2. 提取文本：前后20页 ==========
def extract_front_back_text(pdf_path, front_n=10, back_n=10):
    try:
        doc = fitz.open(pdf_path)
        texts = [doc[i].get_text() for i in range(min(front_n, len(doc)))]
        texts += [doc[i].get_text() for i in range(max(0, len(doc) - back_n), len(doc))]
        doc.close()
        full_text = "\n".join(texts)
        if len(full_text.strip()) < 50:
            raise ValueError("Too little text")
        return full_text
    except Exception as e:
        print(f"⚠️ PyMuPDF failed on {pdf_path.name}, switching to OCR...")
        return extract_text_with_ocr(pdf_path)

# ========== 3. OCR 补救 ==========
def extract_text_with_ocr(pdf_path, dpi=300):
    with tempfile.TemporaryDirectory() as path:
        images = convert_from_path(pdf_path, dpi=dpi, output_folder=path)
        text_parts = [pytesseract.image_to_string(img) for img in images[:3] + images[-3:]]
        return "\n".join(text_parts)

# ========== 4. 构造 Prompt ==========
def build_report_year_prompt(text):
    return f"""
You are an assistant helping extract the reporting year or fiscal year from the following corporate report excerpt.

1. Look for expressions such as:
   - "For the year ended 31 March 2022"
   - "Reporting period: April 2021 – March 2022"
   - "FY2020", etc.
2. If found, return this JSON:
{{
  "report_year": "April 2021 – March 2022",
  "source": "Page 2, main text"
}}

If not found:
{{
  "report_year": null,
  "source": "NOT FOUND"
}}

Report text:
{text}
"""

# ========== 5. Vision 模型辅助 ==========
def encode_image_to_base64(pil_image):
    with tempfile.NamedTemporaryFile(suffix=".png") as f:
        pil_image.save(f.name, format="PNG")
        with open(f.name, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

def extract_year_from_vision(pdf_path, client, page_limit=3):
    try:
        images = convert_from_path(pdf_path, dpi=200)
        for i, img in enumerate(images[:page_limit]):
            b64 = encode_image_to_base64(img)
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "user", "content": [
                        {"type": "text", "text": f"Please extract the fiscal year or reporting period from page {i+1}."},
                        {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{b64}"}}
                    ]}
                ],
                max_tokens=300
            )
            result = response.choices[0].message.content
            if "not found" not in result.lower():
                return {"report_year": result.strip(), "source": f"Vision page {i+1}"}
    except Exception as e:
        return {"report_year": None, "source": f"Vision ERROR: {e}"}
    return {"report_year": None, "source": "Vision NOT FOUND"}

# ========== 6. 主函数：先文本，再 Vision ==========
def extract_report_year(pdf_path, client):
    try:
        text = extract_front_back_text(pdf_path)
        prompt = build_report_year_prompt(text)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=300
        )
        content = response.choices[0].message.content
        parsed = json.loads(content) if content.startswith("{") else eval(content)
        if parsed.get("report_year"):
            return parsed
    except Exception as e:
        print(f"⚠️ GPT-4.1-mini failed on {Path(pdf_path).name}, fallback to Vision...")
    return extract_year_from_vision(pdf_path, client)
from joblib import Parallel, delayed

# 多线程处理函数
def process_pdf(pdf_path):
    try:
        out = extract_report_year(str(pdf_path), client)
        out["filename"] = pdf_path.name
    except Exception as e:
        out = {"filename": pdf_path.name, "report_year": None, "source": f"ERROR: {e}"}
    return out

def batch_extract_years_multithread(pdf_dir, output_csv="results/extracted_report_years_mini.csv", n_jobs=2):
    pdf_dir = Path(pdf_dir)
    pdf_files = sorted(pdf_dir.glob("*.pdf"))
    os.makedirs(Path(output_csv).parent, exist_ok=True)

    # ✅ 并行处理
    results = Parallel(n_jobs=n_jobs, prefer="threads")(
        delayed(process_pdf)(pdf) for pdf in tqdm(pdf_files, desc="📄 Processing PDFs")
    )

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Extraction complete! Results saved to: {output_csv}")

# ========== 8. 运行 ==========
if __name__ == "__main__":
    batch_extract_years_multithread("pdf_folder")  

⚠️ GPT-4.1-mini failed on AAK_AB_aak-sustainability-report-2012-2013_8y6o3xp0.pdf, fallback to Vision...
⚠️ GPT-4.1-mini failed on 888_Holdings_888_Holdings_PLC_Annual_Report__Accounts_2022_4ya4gkz2.pdf, fallback to Vision...


⚠️ GPT-4.1-mini failed on ABC_Holdings_Limited_P020160316609968328069_w68f32zb.pdf, fallback to Vision...
⚠️ GPT-4.1-mini failed on AFLAC_Inc_2013-csr-report-final_oxgizar1.pdf, fallback to Vision...


⚠️ GPT-4.1-mini failed on AIA_Engineering_Ltd_ANNUALREPORT19_20_7ta9u15v.pdf, fallback to Vision...
⚠️ GPT-4.1-mini failed on AIA_Group_Ltd_aia-annual-report-2015-eng_6dpw11ja.pdf, fallback to Vision...


⚠️ GPT-4.1-mini failed on AIB_Group_PLC_annual-financial-report-2010_nzf6dpw1.pdf, fallback to Vision...
⚠️ GPT-4.1-mini failed on AIB_Group_PLC_annual-report-2013_w2uuexld.pdf, fallback to Vision...


⚠️ GPT-4.1-mini failed on AKSA_A40C80A20FF2492CAC2C9CD3A2967D88_u2o5uhav.pdf, fallback to Vision...


...

📄 Processing PDFs:  92%|█████████▏| 1174/1277 [34:38<04:43,  2.75s/it]

⚠️ GPT-4.1-mini failed on Unknown_23076_Whitbread_AR2020_web_0v2mxh4f.pdf, fallback to Vision...


⚠️ GPT-4.1-mini failed on Unknown_adbi-managing-transition-low-carbon-economy_087is5zy.pdf, fallback to Vision...
⚠️ GPT-4.1-mini failed on Unknown_adp07-sus-fr_95qx6prh.pdf, fallback to Vision...


📄 Processing PDFs: 100%|██████████| 1277/1277 [38:47<00:00,  1.82s/it]



✅ Extraction complete! Results saved to: results/extracted_report_years_mini.csv


In [3]:
import pandas as pd

# 读取文件
anno_df = pd.read_excel("check/rfyear_annotation.xlsx")
result_df = pd.read_csv("output/report_years_extracted_new.csv")

# 清洗文件名列
anno_df["pdf_name_clean"] = anno_df["pdf_name"].str.strip().str.lower()
result_df["filename_clean"] = result_df["filename"].str.strip().str.lower()

# 合并两个表
merged_df = pd.merge(
    anno_df,
    result_df,
    left_on="pdf_name_clean",
    right_on="filename_clean",
    how="left"
)

# 年份对比函数
def match_year(human, gpt):
    if pd.isna(human) or pd.isna(gpt):
        return "missing"
    return "match" if str(human).strip().lower() == str(gpt).strip().lower() else "mismatch"

# 添加匹配结果
merged_df["match_result"] = merged_df.apply(
    lambda row: match_year(row["chosen_rfyear"], row["report_year_vision"]),
    axis=1
)

# 仅保留有用字段
final_df = merged_df[[
    "pdf_name", "chosen_rfyear", "report_year_vision", "match_result"
]]

# 保存结果
final_df.to_csv("rfyear_comparison_results_minimal.csv", index=False)

# 输出统计
print("✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv")
print("📊 匹配情况统计：")
print(final_df["match_result"].value_counts())

✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv
📊 匹配情况统计：
match_result
missing     386
mismatch    215
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. 读取两个文件
gpt_df = pd.read_csv("results/extracted_report_years_mini.csv")  # 提取结果
label_df = pd.read_excel("check/rfyear_annotation.xlsx")         # 人工标注文件

# 2. 重命名列以便对齐合并
gpt_df.rename(columns={"filename": "pdf_name", "report_year": "report_year_pred"}, inplace=True)

# 3. 合并两个表（inner join，只保留两个都有的 pdf）
merged = pd.merge(label_df, gpt_df, on="pdf_name", how="inner")

# 4. 标准化字符串格式（去除空格大小写等）
merged["report_year_pred"] = merged["report_year_pred"].astype(str).str.strip().str.lower()
merged["chosen_rfyear"] = merged["chosen_rfyear"].astype(str).str.strip().str.lower()

# 5. 定义匹配逻辑（完全匹配即可）
merged["match"] = merged["report_year_pred"] == merged["chosen_rfyear"]

# 6. 统计指标
total = len(merged)
correct = merged["match"].sum()
accuracy = correct / total

print(f"✅ 合并样本数：{total}")
print(f"✅ 匹配正确数：{correct}")
print(f"✅ 准确率（accuracy）：{accuracy:.2%}")

# 7. 可选：保存对比结果
merged[["pdf_name", "chosen_rfyear", "report_year_pred", "match"]].to_csv("eval/year_comparison.csv", index=False)

✅ 合并样本数：224
✅ 匹配正确数：78
✅ 准确率（accuracy）：34.82%
